<a href="https://colab.research.google.com/github/balados85/deep-learning-brain-mri-classification/blob/main/MRIcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNN Model**


**Importing Libraries**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
tf.__version__

'2.5.0'

**Data Preprocessing**

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
train_set = test_datagen.flow_from_directory('drive/MyDrive/mri_keras_images/train',
                                            target_size = (224, 224),
                                            batch_size = 32)

Found 1746 images belonging to 3 classes.


In [5]:
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_set = validation_datagen.flow_from_directory('drive/MyDrive/mri_keras_images/validation',
                                            target_size = (224, 224),
                                            batch_size = 32)

Found 806 images belonging to 3 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('drive/MyDrive/mri_keras_images/test',
                                            target_size = (224, 224),
                                            batch_size = 32)

Found 569 images belonging to 3 classes.


In [7]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

**Bulding the CNN Model**

In [9]:
cnn = tf.keras.models.Sequential()

**Step 1 : Convolution**

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))

**Step 2 : Pooling**

In [11]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

**Adding a second convolution Layer**

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Dropout(0.25))

**Step 3: Flattening**

In [14]:
cnn.add(tf.keras.layers.Flatten())

**Step 4: Full Connection**

In [15]:
cnn.add(tf.keras.layers.Dense(units=320, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))


**Step 5 Output Layer**

In [15]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

# Training the CNN:

**Compiling the CNN **

In [16]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 32)       

**Training the CNN on the training set and evaluate it on the test set**

In [19]:
cnn.fit(x = train_set, validation_data = validation_set, epochs = 23)

Epoch 1/23
55/55 [==============================] - 1956s 36s/step - loss: 0.8980 - accuracy: 0.5590 - val_loss: 0.3049 - val_accuracy: 0.9578
Epoch 2/23
55/55 [==============================] - 127s 2s/step - loss: 0.2122 - accuracy: 0.9324 - val_loss: 0.0504 - val_accuracy: 0.9801
Epoch 3/23
55/55 [==============================] - 127s 2s/step - loss: 0.0664 - accuracy: 0.9777 - val_loss: 0.1000 - val_accuracy: 0.9826
Epoch 4/23
55/55 [==============================] - 128s 2s/step - loss: 0.0376 - accuracy: 0.9880 - val_loss: 0.0814 - val_accuracy: 0.9826
Epoch 5/23
55/55 [==============================] - 133s 2s/step - loss: 0.0449 - accuracy: 0.9880 - val_loss: 0.1628 - val_accuracy: 0.9677
Epoch 6/23
55/55 [==============================] - 131s 2s/step - loss: 0.0321 - accuracy: 0.9908 - val_loss: 0.0968 - val_accuracy: 0.9789
Epoch 7/23
55/55 [==============================] - 130s 2s/step - loss: 0.0216 - accuracy: 0.9943 - val_loss: 0.1666 - val_accuracy: 0.9777
Epoch 8/23


# Making a single prediction

In [17]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('drive/MyDrive/mri_keras_images/test/2/1010.png', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
train_set.class_indices
if result[0][0] ==0:
        prediction = 'glioma'
elif result[0][0] ==1:
       prediction = 'meningioma'
elif result[0][0] ==2:
       prediction = 'pituitary'
else:
  prediction = 'not Tumor'

In [18]:
print(prediction)

glioma
